# Generate daily bars using dedicated data service with or without corporate actions adjustments  - Python

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot *`daily bars`*.<br>

It enables to retrieve **on-demand** daily bars data by calling a dedicated service on a `daily data store`. 

### Inputs/outputs
Bars sample requires instrument's identifier as per input. It returns open-high-low-close (OHLC) and volume.
This sample shows the differences when adjusting the bars with corporate actions.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyBarsService*: to directly retrieve daily bars objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *mplfinance* as per display package

***

# Run intraday bars sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas mplfinance plotly systemathics.apis

In [ ]:
import os
import csv
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_bars_pb2 as daily_bars
import systemathics.apis.services.daily.v1.daily_bars_pb2_grpc as daily_bars_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily bars* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyBarsService* in order to retrieve daily data

In [ ]:
# set the instrument
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Requests creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# Non adjusted
request = daily_bars.DailyBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    adjustment = False
)

In [ ]:
try:
    # open a gRPC channel
    with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
        credentials = grpc.ssl_channel_credentials(f.read())
    with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
        
        # instantiate the daily prices service
        service = daily_bars_service.DailyBarsServiceStub(channel)
        
        # process the request
        response = service.DailyBars(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
# With adjustments
request_adjusted = daily_bars.DailyBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    adjustment = True
)

In [ ]:
try:
    # open a gRPC channel
    with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
        credentials = grpc.ssl_channel_credentials(f.read())
    with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
        
        # instantiate the daily prices service
        service = daily_bars_service.DailyBarsServiceStub(channel)
        
        # process the request
        response_adjusted = service.DailyBars(
            request = request_adjusted, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
print("Total bars retrieved without adjustments: ", len(response.data))
print("Total bars retrieved with adjustments: ", len(response_adjusted.data))

### Step 4: Export data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
#pPrepare the data frame content
dates=[datetime(b.date.year, b.date.month, b.date.day) for b in response.data]
opens = [b.open for b in response.data]
highs = [b.high for b in response.data]
lows = [b.low for b in response.data]
closes = [b.close for b in response.data]
volumes = [b.volume for b in response.data]

opens_adjusted = [b.open for b in response_adjusted.data]
highs_adjusted = [b.high for b in response_adjusted.data]
lows_adjusted = [b.low for b in response_adjusted.data]
closes_adjusted = [b.close for b in response_adjusted.data]
volumes_adjusted = [b.volume for b in response_adjusted.data]

d = {'Date': dates, 'Open': opens, 'High': highs, 'Low' : lows,'Close': closes, 'Volume': volumes, 'Open_Adjusted': opens_adjusted, 'High_Adjusted': highs_adjusted, 'Low_Adjusted' : lows_adjusted,'Close_Adjusted': closes_adjusted, 'Volume_Adjusted': volumes_adjusted }

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)
df

In [ ]:
filename = 'Export/{0}-{1}-dailybars-comparison.csv'.format(ticker, exchange)
df.to_csv(filename, index=False)

### 5 Plot bars in both cases
The following code snippets display daily bars' request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

#### 5.1 Plot non adjusted daily bars

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Set date as index
df = df.set_index('Date')

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_layout(title = 'Daily bars non-adjusted for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()

#### 5.2 Plot daily bars adjusted with corporate actions

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Candlestick(x=df.index, open=df['Open_Adjusted'], high=df['High_Adjusted'], low=df['Low_Adjusted'], close=df['Close_Adjusted'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume_Adjusted'], name="Volume"), secondary_y=False)

fig.update_layout(title = 'Daily bars with corporate adjustements for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()